### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime

# Suppress all warnings
warnings.filterwarnings("ignore")

# Import User Modules
from utils import load_data

### Load in the Data

In [2]:
df = load_data()
filtered_df = df[[
    "name", "start_date", "type", "sport_type",
    "distance", "total_elevation_gain", "elev_high", "elev_low"
]]

### Hardcoded adjustments for activities with mixed types

In [3]:
activities_distance_divisors = {
    "Ride": 4,
    "MountainBikeRide": 2.5,
    "GravelRide": 3,
    "Run": 1,
    "TrailRun": 1,
    "NordicSki": 1,
    "Hike": (4/3)
}

activities_elevation_divisors = {
    "Ride": 1,
    "MountainBikeRide": 1,
    "GravelRide": 1,
    "Run": 1,
    "TrailRun": 1,
    "NordicSki": 1,
    "Hike": (4/3)
}

partial_trail_run_adjustment = {
    "Mt. Teneriffe + Mt. Si": 0.25,
    "Spray Park Loop": 0.5,
    "Oyster Dome Loop": 0.75,
    "Cutthroat Pass": 0.25,
    "Trappers Peak / Thornton Lakes": 0.25,
    "Trap Pass": 0.25,
    "Mailbox Peak": 0.5,
    "Goat Lake": 0.75,
    "Green Mountain": 0.75
}

def update_ptra(partial_trail_run_adjustment):
        updated_ptra = {}
        for key, value in partial_trail_run_adjustment.items():
            if value == 0.25:
                # if the value is 0.25, this means it is designated as a hike and has already been scaled, so we need to unscale
                updated_ptra[key] = (4/3)*(3/4 + value/4)
            else:
                # other values indicate that this is categorized as a TrailRun, meaning we need to scale down.
                updated_ptra[key] = (3/4 + value/4)
        return updated_ptra

updated_partial_trail_run_adjustment = update_ptra(partial_trail_run_adjustment)

partial_gravel_ride_adjustment = {
     "BLOM / Island Lake": 2.5/(0.25*2.5 + 0.75*3),
     "Island Lake": 2.5/(0.25*2.5 + 0.75*3),
     "Waterloo + DTE": 2.5/(0.25*2.5 + 0.75*3)
}

def convert_timestamp(timestamp):
     return (datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ")).strftime("%m/%d/%y")


elevation_to_capacity = {
    0: 1.0,
    1000: 0.992,
    2000: 0.983,
    3000: 0.972,
    4000: 0.959,
    5000: 0.944,
    6000: 0.927,
    7000: 0.907,
    8000: 0.886,
    9000: 0.863,
    10000: 0.837,
    11000: 0.809,
    12000: 0.78,
}
     

### Compute the difficulty score

In [4]:
# Compute the difficulty score by calculting the distance and elevation contributions with the sport type adjustments
#Start by removing ski
filtered_activities = filtered_activities[filtered_activities['sport_type'] != 'AlpineSki']
filtered_activities["distance_score"] = (df["distance"]*0.000621371)/ df["sport_type"].map(lambda x: activities_distance_divisors.get(x, 1))
filtered_activities["distance_score"] = (filtered_activities["distance_score"]*df["name"].map(lambda x: partial_gravel_ride_adjustment.get(x, 1))).round(2)

filtered_activities["elevation_score"] = (df["total_elevation_gain"]*3*3.28084*0.001)/ df["sport_type"].map(lambda x: activities_elevation_divisors.get(x, 1))
filtered_activities["difficulty_score"] = filtered_activities["distance_score"] + filtered_activities["elevation_score"]
filtered_activities["difficulty_score_without_elevation"] = (filtered_activities["difficulty_score"]*df["name"].map(lambda x: updated_partial_trail_run_adjustment.get(x, 1))).round(2)

#Add adjustments for the average elevation of the activity
filtered_activities["average_elevation"] = (((filtered_activities["elev_high"] + filtered_activities["elev_low"])/2)*3.28084).round(2)
filtered_activities["performance_capacity"] = (filtered_activities["average_elevation"].round(-3)).map(elevation_to_capacity)
filtered_activities["difficulty_score"] = (filtered_activities["difficulty_score_without_elevation"] / filtered_activities["performance_capacity"]).round(2)

# Convert raw distance, elevation, and date to readable formats
filtered_activities["Distance (miles)"] = (filtered_activities["distance"]*0.000621371).round(2)
filtered_activities["Total Elevation Gain (ft)"] = (filtered_activities["total_elevation_gain"]*3.28084).round(0).astype(int)
filtered_activities["Date"] = filtered_activities["start_date"].apply(convert_timestamp)

filtered_activities["elev_high"] = filtered_activities["elev_high"]*3.28084
# Extract only cleaned columns
cleaned_activities = filtered_activities[["name", "Date", "sport_type", "Distance (miles)", "Total Elevation Gain (ft)", "elev_high","average_elevation", "performance_capacity", "difficulty_score_without_elevation", "difficulty_score"]]

NameError: name 'filtered_activities' is not defined

### Filter and analyze output data

In [ ]:
#filtered_activities[filtered_activities["sport_type"].isin(["TrailRun", "Hike"])].sort_values(by="difficulty_score", ascending=False).head(40)
hardest_hiking_activities = cleaned_activities[cleaned_activities["sport_type"].isin(["TrailRun", "Hike"])].sort_values(by="difficulty_score", ascending=False).reset_index(drop=True)
hardest_biking_activities = cleaned_activities[cleaned_activities["sport_type"].isin(["MountainBikeRide", "GravelRide", "Ride"])].sort_values(by="difficulty_score", ascending=False).reset_index(drop=True)
hardest_running_activities = cleaned_activities[cleaned_activities["sport_type"].isin(["TrailRun", "Run"])].sort_values(by="difficulty_score", ascending=False).reset_index(drop=True)
hardest_offroad_riding_activities = cleaned_activities[cleaned_activities["sport_type"].isin(["MountainBikeRide", "GravelRide"])].sort_values(by="difficulty_score", ascending=False).reset_index(drop=True)
hardest_overall_activities = cleaned_activities.sort_values(by="difficulty_score", ascending=False).reset_index(drop=True)